### CoreBx_island - Try to process entire N. Core Banks

Interpolate the North Core Banks DEMs onto rotated 1-m grid and save each as a .nc file.

New in v2
* Now 4D maps, two made made during visit to Santa Cruz and two ftp'd from Andy
* Apr. 9 - changed to _v3 for Sep map
* Now does the interpolation without the loop
* Apr. 21 - moved origin to SE to accomodate curvature in NE end of island. Add 400 m to size of array.
* Watch file names, esp. underline (or not) after "1m_DEM"

TODO: The alongshore/cross-shore names are switched.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
# from dask.distributed import LocalCluster
from scipy import interpolate, signal
%matplotlib inline

In [2]:
# define all of the functions by runnng this python file
%run -i CoreBx_funcs

In [3]:
def make_grid(name=None,e0=None,n0=None,xlen=None,ylen=None,dxdy=None,theta=None):
    nx = int((1./dxdy)*xlen)
    ny = int((1./dxdy)*ylen)

    xcoords = np.linspace(0.5*dxdy,xlen-0.5*dxdy,nx)
    ycoords = np.linspace(0.5*dxdy,ylen-0.5*dxdy,ny)

    # these will be the coordinates in rotated space
    xrot, yrot = np.meshgrid(xcoords, ycoords ,sparse=False, indexing='xy')

    print('Shape of xrot, yrot: ',np.shape(xrot),np.shape(yrot))
    shp = np.shape(xrot)
    xu, yu = box2UTMh(xrot.flatten(), yrot.flatten(), e0, n0, theta)
    xu=np.reshape(xu,shp)
    yu=np.reshape(yu,shp)
    # write the UTM coords of the corners to an ASCII file
    corners = np.asarray(  [[xu[0][0],yu[0][0]],\
                           [xu[0][-1],yu[0][-1]],\
                           [xu[-1][-1],yu[-1][-1]],\
                           [xu[-1][0],yu[-1][0]],\
                           [xu[0][0],yu[0][0]]])

    print(corners)
    fn = name+'.csv'
    np.savetxt(fn, corners, delimiter=",")
    return xu, yu, xrot, yrot, xcoords, ycoords

In [4]:
# April 9, 2020: Replaced   "2019-09-12-13_1m_DEM_4D_crop.tif",\
# with _v3 and re-ran on my desktop

fdir = "C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/"
#fdir = "D:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/"


fnames = (\
          "2019-08-30_1m_DEM_4D_crop2.tif",\
          "2019-09-12-13_1mDEM_4D_v3.tif",\
          "2019-10-11_1m_DEM_4D_crop.tif",\
          "2019-11-26_1m_DEM_4D_crop.tif")

titles = ([\
         "8/30/2020 pre-Dorian",\
         "9/12-13/2020 post-Dorian",\
          "10/11/2020",\
          "11/26 post-Nor'easter"])

nf = len(fnames)

fill_fnames = ('EBK_201909_YesLidar_Comb_Extent.tif')
fill_titles = ('Sep_fill')

# optional median-filter smoothing of original maps
smooth = False
# kernal size...this should be an odd number >= dxy/0.1
ksize = 3

In [5]:
# Make an array of dicts, where analysis region is defined by:
#  name
#  e0 - UTM Easting of origin [m]
#  n0 - UTM Northing of origin [m]
#  xlen - Length of alongshore axis [m]
#  ylen - Length of cross-shore axis [m]
#  dxdy - grid size (must be isotropic right now) [m]
#  theta - rotation CCW from x-axis [deg]

r = {'name':"ncorebx","e0": 378500.,"n0": 3856350.,"xlen": 36000.,"ylen": 1100.,"dxdy": 1.,"theta": 42.}

In [6]:
# move the origin 400 m SE
xo,yo = xycoord(400.,42.+90)
print(xo,yo)
r['e0']=r['e0']+xo
r['n0']=r['n0']+yo

# add 400 m to ylen
r['ylen']=r['ylen']+400.

297.2579301909577 -267.6522425435433


In [7]:
r

{'name': 'ncorebx',
 'e0': 378797.25793019094,
 'n0': 3856082.3477574564,
 'xlen': 36000.0,
 'ylen': 1500.0,
 'dxdy': 1.0,
 'theta': 42.0}

In [8]:
print(r['name'])
xu,yu,xrot,yrot,xcoords,ycoords = make_grid(**r)
ny,nx = np.shape(xu)
print(ny,nx)

ncorebx
Shape of xrot, yrot:  (1500, 36000) (1500, 36000)
[[ 378797.2949373  3856083.05389517]
 [ 405549.76550966 3880171.08659348]
 [ 404546.73873073 3881285.06068688]
 [ 377794.26815837 3857197.02798856]
 [ 378797.2949373  3856083.05389517]]
1500 36000


In [9]:
%%time
# Read in the fill map and make netcdf files
fn = fdir+fill_fnames
print(fn)

# open the tif with XArray as a DataArray
daf = xr.open_rasterio(fn)

print( np.shape(np.flipud(daf['y'].values)), np.shape(daf['x'].values), np.shape( np.flipud(daf.values)) )
x = daf['x'].values
y = np.flipud(daf['y'].values)

# Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
# However, make sure to squeeze before flipping
z = np.flipud(np.squeeze(daf.values))
print(np.shape(x),np.shape(y),np.shape(z))

f = interpolate.RegularGridInterpolator( (y, x), z, method='linear')   

# Array for interpolated elevations
zi=np.NaN*np.ones((ny,nx))

# this is a slow iteration through all of the points, but allows us to skip ones that are outside
for ij in np.ndindex(zi.shape):
    try:
        zi[ij]=f((yu[ij],xu[ij]))
    except:
        zi[ij]=np.NaN

# this is the fast technique.
# zi=f((yu,xu))

da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
da = da.chunk()

fno = r['name']+'_Sep_fill_v4.nc'
da.to_netcdf(fno)

C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/EBK_201909_YesLidar_Comb_Extent.tif
(23957,) (27421,) (1, 23957, 27421)
(27421,) (23957,) (23957, 27421)
Wall time: 1h 48min 39s


In [10]:
%%time
dslist=[]
for i in range(nf):
    fn = fdir+fnames[i]
    print(i, fn)

    # open the tif with XArray as a DataArray
    da = xr.open_rasterio(fn)

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    if(smooth):
        # smooth with 2D running median
        zs = signal.medfilt2d(z, kernel_size=ksize)
    else:
        zs = z

    f = interpolate.RegularGridInterpolator( (y, x), zs, method='linear')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    for ij in np.ndindex(zi.shape):
        try:
            zi[ij]=f((yu[ij],xu[ij]))
        except:
            zi[ij]=np.NaN

    # this is the fast iteration, which only works when all of the source points fall inside the target box
    #   zi=f((yu,xu))

    da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    da = da.chunk()
    dslist.append(da)

dsa = xr.concat(dslist, dim='map')
fn = r['name']+'_v4.nc'
dsa.to_netcdf(fn)

0 C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/2019-08-30_1m_DEM_4D_crop2.tif
(24290,) (27666,) (1, 24290, 27666)
(27666,) (24290,) (24290, 27666)
1 C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/2019-09-12-13_1mDEM_4D_v3.tif
(24035,) (27493,) (1, 24035, 27493)
(27493,) (24035,) (24035, 27493)
2 C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/2019-10-11_1m_DEM_4D_crop.tif


RasterioIOError: C:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/clipped_dems/2019-10-11_1m_DEM_4D_crop.tif: No such file or directory